In [2]:
#Password Vault
#Kevin Lochard
#Student ID: 1160568
#Password Vault with Hashing Version 1.6

#importing our DB for passowrds and hashing the passwords
import sqlite3, hashlib
from tkinter import * # makes the UI popup to interact with
from tkinter import simpledialog # this is going to be used when entering new passwords inside the vault
from functools import partial

#database code
with sqlite3.connect('simple_Banking_password_vault.db') as db:
    cursor = db.cursor() #initates the db
#Creating table in the DB                     
cursor.execute("""
CREATE TABLE IF NOT EXISTS masterpassword(
id INTERGER PRIMARY KEY,
password TEXT NOT NULL);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS vault(
id INTEGER PRIMARY KEY,
website TEXT NOT NULL,
username TEXT NOT NULL,
password TEXT NOT NULL);
""")

#Create PopUp and ask for a string
def popUp(text):
    answer = simpledialog.askstring("input string", text)
    
    return answer


#used to save info into db table
window = Tk()

window.title("Simple Banking Safety Deposit Box / Password Vault") #title of pop up page

#hashing Script
def hashPassword(input):
    hash = hashlib.md5(input) #library that hasehes passwords
    hash = hash.hexdigest()
    
    return hash

def firstTimeScreen():
    window.geometry("500x250")
    lbl = Label(window, text = "Welcome to the Virtual Vault, Please Create a Master Password:")
    lbl.config(anchor=CENTER)
    lbl.pack()
    
    txt = Entry(window, width= 20, show = "*") #hide password as it is typed
    txt.pack()
    txt.focus() # when launhed cursor is already in place
    
    lbl1 = Label(window, text = "Please Re-Enter The Password:")
    lbl1.config(anchor=CENTER)
    lbl1.pack()
    
    txt1 = Entry(window, width= 20, show = "*") #hide password as it is typed
    txt1.pack()

    def savePassword():
        if txt.get() == txt1.get():
            hashedPassword = hashPassword(txt.get().encode('utf-8'))# endcoding it to so we cant read it
                     
            #adds to the db the password when the user presses submit        
            insert_password = """INSERT INTO masterpassword(password)
            VALUES(?) """
            cursor.execute(insert_password, [(hashedPassword)])
            db.commit() #saving it to the DB
                     
            passwordVault()# when we save password takes us to the vault
        else:
            lbl.config(text="Passwords Do NOT Match, Please Try again")
            txt.delete(0, 'end') #Erases the passowrd when you get it wrong
            
                
     #Button on screen to save info       
    btn = Button(window, text="Save & Submit", command=savePassword)
    btn.pack(pady=5)
    
def loginScreen():
    for widget in window.winfo_children():
        widget.destroy()
    window.geometry("500x250")
    
    lbl = Label(window, text = "Welcome to the Virtual Vault, Please Enter The Master Password: ")
    lbl.config(anchor=CENTER)
    lbl.pack()
    
    txt = Entry(window, width= 20, show = "*") #hide password as it is typed
    txt.pack()
    txt.focus() #when launched cursor is already in place
    
    lbl1 = Label(window)
    lbl1.config(anchor=CENTER)
    lbl1.pack(side=TOP)
    
    def getMasterPassword():
        checkHashedPassword = hashPassword(txt.get().encode('utf-8'))
        cursor.execute('SELECT * FROM masterpassword WHERE password = ?', [(checkHashedPassword)])
        return cursor.fetchall()
    
    def checkPassword():
        password = getMasterPassword()
        
        if password:
            print("Correct Passwword")
            passwordVault() #takes you inside the vault if the password is correct
        else:
            txt.delete(0, 'end') #Erases the passowrd when you get it wrong
            lbl1.config(text="Incorrect Password Please Try Again")
    
    btn = Button(window, text="Submit", command=checkPassword) #ChecksPassword agaisnt database
    btn.pack(pady=5)
             
def passwordVault():
    for widget in window.winfo_children(): 
    #destroys text behind to make it cleaner and takes you to the actual vault
        widget.destroy()
        
        #button alowing you to add entry to the vault
        def addEntry():
            text1 = "Please Enter Website Name:"
            text2 = "Please Enter Username:"
            text3 = "Please Enter Password:"
            #will be adding encryption to this portion in next version
            website = (popUp(text1))
            username = (popUp(text2))
            password = (popUp(text3))

        #inserting into our table
            insert_fields = """INSERT INTO vault(website, username, password) 
            VALUES(?, ?, ?) """
            cursor.execute(insert_fields, (website, username, password))
            db.commit()

            passwordVault() #so it refreshes

    def removeEntry(input):
        cursor.execute("DELETE FROM vault WHERE id = ?", (input,))
        db.commit()
        passwordVault() #so it refreshes

        
        
    window.geometry("950x350") #Actual vault page sizing
    window.resizable(height=None, width=None)                
    lbl = Label(window, text = "Welcome to the Virtual Vault Provided by Simple Banking")
    lbl.config(anchor=CENTER)
    lbl.grid(column=1)
    
    #this button calls back the add entry function
    btn = Button(window, text="Add Entry", command=addEntry)
    btn.grid(column=1, pady=10)

    #keeping the same row but changing collums for inserting
    lbl = Label(window, text="Website")
    lbl.grid(row=2, column=0, padx=80)
    lbl = Label(window, text="Username")
    lbl.grid(row=2, column=1, padx=80)
    lbl = Label(window, text="Password")
    lbl.grid(row=2, column=2, padx=80)
    
    cursor.execute('SELECT * FROM vault') # when using fetchall the line before it has to be execute
    if (cursor.fetchall() != None): # to make sure there is something in the table
        i = 0
        while True: #runs until broken
            cursor.execute('SELECT * FROM vault')
            array = cursor.fetchall()

            if (len(array) == 0):
                break #broken here
#will add encryption and decription later
            lbl1 = Label(window, text=((array[i][1], )), font=("Helvetica", 12))
            lbl1.grid(column=0, row=(i+3)) #checks first entry; take the website text, then starts on row 3 then keeps going up
            lbl2 = Label(window, text=((array[i][2], )), font=("Helvetica", 12))
            lbl2.grid(column=1, row=(i+3))
            lbl3 = Label(window, text=((array[i][3], )), font=("Helvetica", 12))
            lbl3.grid(column=2, row=(i+3))

            btn = Button(window, text="Delete", command=  partial(removeEntry, array[i][0])) #take function with current array and id and keeps it.
            btn.grid(column=3, row=(i+3), pady=10)

            i = i +1

            cursor.execute('SELECT * FROM vault') #stopping loop to not run forever
            if (len(cursor.fetchall()) <= i):
                break
    
#checking db and deciding if you are a first time user or not              
cursor.execute('SELECT * FROM masterpassword')
if (cursor.fetchall()):
    loginScreen()
else:
    firstTimeScreen()
window.mainloop()

Correct Passwword
